In [1]:
import sys
sys.path.append('../unseen')

import myfuncs
import bias_correction

In [2]:
fcst_file = '/g/data/xv83/dbi599/pr_cafe-c5-d60-pX-f6_19911101-19921101_3650D_cafe-grid-TAS-POINT.zarr.zip'
obs_file = '/g/data/xv83/dbi599/pr_awap_cafe-grid-TAS-POINT.zarr.zip'
output_file = '/g/data/xv83/dbi599/pr_cafe-c5-d60-pX-f6_awap-additive-correction_19911101-19921101_3650D_cafe-grid-TAS-POINT.zarr.zip'

In [5]:
ds_fcst = myfuncs.open_file(fcst_file)
ds_fcst

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 2, lead_time: 3650)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) object 1991-11-01 00:00:00 1992-11-01 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
    time       (lead_time, init_date) object dask.array<chunksize=(3650, 2), meta=np.ndarray>
Data variables:
    pr         (init_date, lead_time, ensemble) float64 dask.array<chunksize=(1, 50, 96), meta=np.ndarray>
Attributes:
    comment:    pressure level interpolator, version 3.0, precision=double
    filename:   atmos_isobaric_daily.zarr
    grid_tile:  N/A
    grid_type:  regular
    history:    Wed Jun 23 11:28:12 2021: /g/data/e14/dbi599/miniconda3/envs/...
    title:      AccessOcean-AM2

In [6]:
da_fcst = ds_fcst['pr']
da_fcst

<xarray.DataArray 'pr' (init_date: 2, lead_time: 3650, ensemble: 96)>
dask.array<open_dataset-c59f57133b39eace4d773b7cd7d29f85pr, shape=(2, 3650, 96), dtype=float64, chunksize=(1, 50, 96), chunktype=numpy.ndarray>
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) object 1991-11-01 00:00:00 1992-11-01 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
    time       (lead_time, init_date) object dask.array<chunksize=(3650, 2), meta=np.ndarray>
Attributes:
    cell_methods:   time: mean
    interp_method:  conserve_order1
    long_name:      Total precipitation rate
    time_avg_info:  average_T1,average_T2,average_DT
    units:          kg/m2/s

In [7]:
init_dates = myfuncs.cftime_to_str(da_fcst['init_date'])
init_dates

['1991-11-01', '1992-11-01']

In [8]:
n_lead_steps = int(da_fcst['lead_time'].values.max()) + 1
print(n_lead_steps)

3650


In [10]:
ds_obs = myfuncs.open_file(obs_file, variables=['pr'])
ds_obs

<xarray.Dataset>
Dimensions:  (time: 44136)
Coordinates:
  * time     (time) object 1900-01-01 09:00:00 ... 2020-12-02 09:00:00
Data variables:
    pr       (time) float64 dask.array<chunksize=(44136,), meta=np.ndarray>
Attributes:
    history:  Tue Jun 22 17:00:09 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [11]:
ds_obs = myfuncs.stack_by_init_date(ds_obs, init_dates, n_lead_steps)
ds_obs

<xarray.Dataset>
Dimensions:    (init_date: 2, lead_time: 3650)
Coordinates:
  * init_date  (init_date) object 1991-11-01 09:00:00 1992-11-01 09:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
    time       (init_date, lead_time) object 1991-11-01 09:00:00 ... 2002-10-...
Data variables:
    pr         (init_date, lead_time) float64 dask.array<chunksize=(2, 3650), meta=np.ndarray>
Attributes:
    history:  Tue Jun 22 17:00:09 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [13]:
da_obs = ds_obs['pr']
da_obs

<xarray.DataArray 'pr' (init_date: 2, lead_time: 3650)>
dask.array<getitem, shape=(2, 3650), dtype=float64, chunksize=(2, 3650), chunktype=numpy.ndarray>
Coordinates:
  * init_date  (init_date) object 1991-11-01 09:00:00 1992-11-01 09:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
    time       (init_date, lead_time) object 1991-11-01 09:00:00 ... 2002-10-...
Attributes:
    analysis_time:                 2010-10-23 14:49:46 UTC
    analysis_version_number:       3.01
    cell_methods:                  time: sum
    frequency:                     daily
    grid_mapping:                  crs
    length_scale_for_analysis:     80.0
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    source:                        Interpolated surface observations
    standard_name:                 lwe_thickness_of_precipitation_amount
    units:                         mm
    valid_range:                   [-1.0, 100000.0]

In [15]:
bias = bias_correction.get_bias(da_fcst, da_obs, 'additive')
bias

<xarray.DataArray 'pr' (month: 1, lead_time: 3650)>
dask.array<sub, shape=(1, 3650), dtype=float64, chunksize=(1, 50), chunktype=numpy.ndarray>
Coordinates:
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
  * month      (month) int64 11
Attributes:
    bias_correction_method:  additive

In [16]:
da_fcst_bc = bias_correction.remove_bias(da_fcst, bias, 'additive')
da_fcst_bc

<xarray.DataArray 'pr' (init_date: 2, lead_time: 3650, ensemble: 96)>
dask.array<sub, shape=(2, 3650, 96), dtype=float64, chunksize=(1, 50, 96), chunktype=numpy.ndarray>
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) object 1991-11-01 00:00:00 1992-11-01 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
    time       (lead_time, init_date) object dask.array<chunksize=(3650, 2), meta=np.ndarray>
Attributes:
    bias_correction_method:  additive

In [17]:
ds_fcst_bc = da_fcst_bc.to_dataset()
infile_logs = {fcst_file: ds_fcst.attrs['history'],
               obs_file: ds_obs.attrs['history']}
ds_fcst_bc.attrs['history'] = myfuncs.get_new_log(infile_logs=infile_logs)
ds_fcst_bc

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 2, lead_time: 3650)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) object 1991-11-01 00:00:00 1992-11-01 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 ... 3644 3645 3646 3647 3648 3649
    time       (lead_time, init_date) object dask.array<chunksize=(3650, 2), meta=np.ndarray>
Data variables:
    pr         (init_date, lead_time, ensemble) float64 dask.array<chunksize=(1, 50, 96), meta=np.ndarray>
Attributes:
    history:  Wed Jun 23 13:32:39 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [18]:
ds_fcst_bc = ds_fcst_bc.chunk({'init_date': -1, 'lead_time': -1})
myfuncs.to_zarr(ds_fcst_bc, output_file)